# Challenge on predicting of resolvability of crimes for BATMAN


### Socio Team  or "Bat"Team 

<i>Goukam , Imad , Mohamed , Nassim (M2 AIC Paris_Sud) </i>

## Introduction
The data set was download in this url:<a>https://data.sfgov.org/Public-Safety/SFPD-Incidents-Current-Year-2015-/ritf-b9ki</a>. 
The data is thin, it contains

* Dates - timestamp of the crime incident
* Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
* Descript - detailed description of the crime incident (only in train.csv)
* DayOfWeek - the day of the week
* PdDistrict - name of the Police Department District
* Resolution - how the crime incident was resolved (only in train.csv)
* Address - the approximate street address of the crime incident 
* X - Longitude
* Y - Latitude




** The goal is to predict the <code>Resolution</code> column. The prediction quality is measured by RMSE**. 



In [189]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## Fetch the data and load it in pandas

In [190]:
import os

In [191]:
local_filename = 'data/original_data.csv'
data = pd.read_csv(local_filename, index_col=0)

In [192]:
data.shape

(153618, 11)

In [193]:
data.columns.values


array(['Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Location'], dtype=object)

In [194]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
IncidntNum,,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,SOUTHERN,NONE,1100 Block of MISSION ST,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,INGLESIDE,"ARREST, BOOKED",0 Block of LELAND AV,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,INGLESIDE,NONE,200 Block of BLYTHDALE AV,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,CENTRAL,NONE,800 Block of POST ST,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


In [195]:
print min(data['Date'])
print max(data['Date'])

01/01/2015
12/31/2015


In [196]:
data['PdDistrict'].unique()

array(['SOUTHERN', 'INGLESIDE', 'CENTRAL', 'BAYVIEW', 'PARK', 'NORTHERN',
       'MISSION', 'TENDERLOIN', 'TARAVAL', 'RICHMOND'], dtype=object)

In [197]:
data['Category'].unique()

array(['ASSAULT', 'VANDALISM', 'OTHER OFFENSES', 'NON-CRIMINAL',
       'LARCENY/THEFT', 'VEHICLE THEFT', 'BURGLARY', 'ROBBERY', 'WARRANTS',
       'SUSPICIOUS OCC', 'WEAPON LAWS', 'DRUNKENNESS', 'TRESPASS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'MISSING PERSON',
       'SECONDARY CODES', 'FRAUD', 'EMBEZZLEMENT',
       'SEX OFFENSES, FORCIBLE', 'BRIBERY', 'STOLEN PROPERTY',
       'DISORDERLY CONDUCT', 'ARSON', 'FAMILY OFFENSES', 'RUNAWAY',
       'DRIVING UNDER THE INFLUENCE', 'KIDNAPPING', 'PROSTITUTION',
       'SUICIDE', 'LIQUOR LAWS', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'SEX OFFENSES, NON FORCIBLE', 'LOITERING',
       'PORNOGRAPHY/OBSCENE MAT', 'TREA'], dtype=object)

In [198]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
IncidntNum,,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,SOUTHERN,NONE,1100 Block of MISSION ST,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,INGLESIDE,"ARREST, BOOKED",0 Block of LELAND AV,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,INGLESIDE,NONE,200 Block of BLYTHDALE AV,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,CENTRAL,NONE,800 Block of POST ST,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


In [199]:
data['Resolution'].unique()

array(['ARREST, BOOKED', 'NONE', 'UNFOUNDED', 'ARREST, CITED',
       'JUVENILE BOOKED', 'EXCEPTIONAL CLEARANCE', 'PSYCHOPATHIC CASE',
       'NOT PROSECUTED', 'LOCATED',
       'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'JUVENILE DIVERTED',
       'JUVENILE CITED', 'COMPLAINANT REFUSES TO PROSECUTE',
       'JUVENILE ADMONISHED'], dtype=object)

In [200]:
data = data.replace(to_replace="ARREST, BOOKED", value=1)
data = data.replace(to_replace="ARREST, CITED", value=1)
data = data.replace(to_replace="NONE", value=0)
data = data.replace(to_replace="UNFOUNDED", value=0)
data = data.replace(to_replace="JUVENILE BOOKED", value=0)
data = data.replace(to_replace="EXCEPTIONAL CLEARANCE", value=0)
data = data.replace(to_replace="PSYCHOPATHIC CASE", value=0)
data = data.replace(to_replace="NOT PROSECUTED", value=0)
data = data.replace(to_replace="LOCATED", value=0)
data = data.replace(to_replace="CLEARED-CONTACT JUVENILE FOR MORE INFO", value=0)
data = data.replace(to_replace="COMPLAINANT REFUSES TO PROSECUTE", value=0)
data = data.replace(to_replace="JUVENILE ADMONISHED", value=0)
data = data.replace(to_replace="JUVENILE DIVERTED", value=0)
data = data.replace(to_replace="JUVENILE CITED", value=0)


*** Remove a "Address" To samplifier***

In [201]:
data = data.drop('Address', 1)
data = data.drop('Location', 1)


In [202]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,X,Y
IncidntNum,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,1,-122.403405,37.775421
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,SOUTHERN,0,-122.411626,37.778590
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,INGLESIDE,1,-122.404263,37.711339
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,INGLESIDE,0,-122.420557,37.710895
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,CENTRAL,0,-122.415844,37.787402


In [203]:
data.dtypes

Category       object
Descript       object
DayOfWeek      object
Date           object
Time           object
PdDistrict     object
Resolution      int64
X             float64
Y             float64
dtype: object

*** Replace a San f PdDistrict with Gotham city district *** (<a>https://en.wikipedia.org/wiki/Gotham_City</a>)

In [204]:
data['PdDistrict'].unique()

array(['SOUTHERN', 'INGLESIDE', 'CENTRAL', 'BAYVIEW', 'PARK', 'NORTHERN',
       'MISSION', 'TENDERLOIN', 'TARAVAL', 'RICHMOND'], dtype=object)

In [205]:
data = data.replace(to_replace="SOUTHERN", value="Otisburg")
data = data.replace(to_replace="INGLESIDE", value="Burnley")
data = data.replace(to_replace="CENTRAL", value="East End")
data = data.replace(to_replace="BAYVIEW", value="Old Gotham")
data = data.replace(to_replace="PARK", value="Robinson Park")
data = data.replace(to_replace="NORTHERN", value="Chinatown")
data = data.replace(to_replace="MISSION", value="Bristol County")
data = data.replace(to_replace="TENDERLOIN", value="The Bowery")
data = data.replace(to_replace="TARAVAL", value="Diamond")
data = data.replace(to_replace="RICHMOND", value="Falcone Penthouse")

In [250]:
data = data.reset_index()
data.head()


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,X,Y
0,160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,Otisburg,1,-122.403405,37.775421
1,166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,Otisburg,0,-122.411626,37.778590
2,160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,Burnley,1,-122.404263,37.711339
3,160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,Burnley,0,-122.420557,37.710895
4,166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,East End,0,-122.415844,37.787402


# Save a new data 

In [251]:
data.to_csv('data/data_set.csv')

### Split data


In [252]:
data_Train, data2  = np.array_split(data,2)
data_Test, data_Valid = np.array_split(data2,2)

# Train

In [253]:
y_train = data_Train.Resolution.reset_index()
x_train = data_Train.drop(["Resolution"],axis = 1)

In [254]:
y_train.to_csv('data/y_train.csv')

In [255]:
x_train.to_csv('data/x_train.csv')

In [248]:
y_train.drop(["IncidntNum"],axis = 1)

,Resolution
0,1
1,0
2,1
3,0
4,0
5,0
6,0
7,0
8,0
9,0


# Validation

In [223]:
y_test = data_Test.Resolution
x_test = data_Test.drop(["Resolution"],axis = 1)

In [224]:
y_test.to_csv('data/y_test.csv')

In [225]:
x_test.to_csv('data/x_test.csv')

# Test

In [226]:
y_valid = data_Valid.Resolution
x_valid = data_Valid.drop(["Resolution"],axis = 1)

In [227]:
y_valid.to_csv('data/y_valid.csv')

In [228]:
x_valid.to_csv('data/x_valid.csv')

In [237]:
y_train

IncidntNum
160000108    1
166004914    0
160000095    1
160038137    0
166002930    0
160002819    0
166002980    0
166008223    0
160001485    0
160000073    0
160000299    0
160002386    0
160000249    0
160000227    0
151127305    0
160000051    0
160000932    0
151127292    0
160001429    0
151127292    0
151127311    1
151127311    1
151127292    0
166002134    0
160004031    0
151127220    1
151127220    1
160000023    0
151127208    1
151127270    0
            ..
150556688    1
150556688    1
150556688    1
150556741    1
150556707    0
150556713    0
150556650    0
150556531    0
150556848    0
156153797    0
150556735    1
150556735    1
150559472    0
150556666    1
150556666    1
150558628    0
150556666    1
156163194    0
150556484    1
150557454    0
156154024    0
150556478    0
156153781    0
156160015    0
150556490    0
150556434    1
150749946    0
150556440    0
150559347    0
156161502    0
Name: Resolution, dtype: int64

In [231]:
y_valid.reset_index()

,IncidntNum,Resolution
0,150277107,0
1,156078082,0
2,156077589,0
3,156080300,0
4,150276820,0
5,150277492,0
6,150292791,0
7,150383986,0
8,150278020,0
9,150276773,1
